In [1]:
import pandas as pd

# URL da tabela 1737 do SIDRA
url = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/all/p/all/d/1/f/u"

# Lê os dados como JSON e transforma em DataFrame
df = pd.read_json(url)

# Ajusta o cabeçalho
df.columns = df.iloc[0]
df = df.drop(index=0)

# Renomeia colunas relevantes
df

,Nível Territorial (Código),Nível Territorial,Unidade de Medida,Valor,Brasil (Código),Brasil,Variável,Mês
1,1,Brasil,Número-índice,0.0,1,Brasil,IPCA - Número-índice (base: dezembro de 1993 =...,dezembro 1979
2,1,Brasil,Número-índice,0.0,1,Brasil,IPCA - Número-índice (base: dezembro de 1993 =...,janeiro 1980
3,1,Brasil,Número-índice,0.0,1,Brasil,IPCA - Número-índice (base: dezembro de 1993 =...,fevereiro 1980
4,1,Brasil,Número-índice,0.0,1,Brasil,IPCA - Número-índice (base: dezembro de 1993 =...,março 1980
5,1,Brasil,Número-índice,0.0,1,Brasil,IPCA - Número-índice (base: dezembro de 1993 =...,abril 1980
...,...,...,...,...,...,...,...,...
3260,1,Brasil,%,4.9,1,Brasil,IPCA - Variação acumulada em 12 meses,novembro 2024
3261,1,Brasil,%,4.8,1,Brasil,IPCA - Variação acumulada em 12 meses,dezembro 2024
3262,1,Brasil,%,4.6,1,Brasil,IPCA - Variação acumulada em 12 meses,janeiro 2025
3263,1,Brasil,%,5.1,1,Brasil,IPCA - Variação acumulada em 12 meses,fevereiro 2025


In [2]:
ts = pd.read_parquet('df_valor_ajuste_contrato.parquet')
ts

,Assets,2025-04-11
0,DAP25,"139,29"
1,DAP26,"236,36"
2,DAP27,"257,15"
3,DAP28,"363,27"
4,DAP29,"567,30"
5,DAP30,"553,02"
6,DAP32,"652,51"
7,DAP33,"643,62"
8,DAP35,"693,58"
9,DAP40,"971,23"


In [ ]:
                df_ajuste = pd.read_parquet('df_valor_ajuste_contrato.parquet')

                # Selecionar apenas as colunas de data (ignorando a primeira, que é "Assets")
                colunas_datas = df_ajuste.columns[1:]

                # Aplicar replace apenas nessas colunas
                df_ajuste[colunas_datas] = df_ajuste[colunas_datas].replace('\.', '', regex=True).replace(',', '.', regex=True)

                # Converter para float
                df_ajuste[colunas_datas] = df_ajuste[colunas_datas].astype(float)

                # Converter colunas que representam datas para datetime (ignorando a coluna de índice "Assets")
                colunas_datas = pd.to_datetime(df_ajuste.columns[1:], errors='coerce')  # Ignora a primeira coluna
                colunas_validas = [df_ajuste.columns[0]] + df_ajuste.columns[1:][colunas_datas >= dia_compra].tolist()

                # Filtrar o DataFrame com as colunas desejadas
                df_ajuste = df_ajuste.loc[:, colunas_validas]

                #Rendimento é igual a linha da Asset que contém o ativo somando as colunas de data
                rendimento = (df_ajuste.loc[df_ajuste['Assets'] == asset].drop(columns='Assets').sum(axis=1).values[0]) * qtd_final

In [ ]:
                df_ajuste = pd.read_parquet('df_valor_ajuste_contrato.parquet')

                # Selecionar apenas as colunas de data (ignorando a primeira, que é "Assets")
                colunas_datas = df_ajuste.columns[1:]
                # Aplicar replace apenas nessas colunas
                df_ajuste[colunas_datas] = df_ajuste[colunas_datas].replace('\.', '', regex=True).replace(',', '.', regex=True)
                # Converter para float
                df_ajuste[colunas_datas] = df_ajuste[colunas_datas].astype(float)
                # Converter strings das colunas de data para datetime
                colunas_datas_dt = pd.to_datetime(colunas_datas, errors='coerce')
                mapa_colunas = dict(zip(colunas_datas_dt, colunas_datas))

                colunas_datas = pd.to_datetime(df_ajuste.columns[1:], errors='coerce')  # Ignora a primeira coluna
                coluna_dia_compra_str = mapa_colunas.get(dia_compra)

                # Filtrar o DataFrame com as colunas desejadas
                valor_ajuste = df_ajuste.loc[df_ajuste['Assets'] == asset, coluna_dia_compra_str].values[0]
                #Rendimento é igual a linha da Asset que contém o ativo somando as colunas de data
                rendimento = valor_ajuste * qtd_final

In [23]:
asset = 'DAP35'
qtd_final = 10
dia_de_compra = pd.to_datetime('2025-04-11')  # Data de compra do contrato
df_ajuste = pd.read_parquet('df_valor_ajuste_contrato.parquet')

# Selecionar apenas as colunas de data (ignorando a primeira, que é "Assets")
colunas_datas = df_ajuste.columns[1:]
# Aplicar replace apenas nessas colunas
df_ajuste[colunas_datas] = df_ajuste[colunas_datas].replace('\.', '', regex=True).replace(',', '.', regex=True)
# Converter para float
df_ajuste[colunas_datas] = df_ajuste[colunas_datas].astype(float)
# Converter strings das colunas de data para datetime
colunas_datas_dt = pd.to_datetime(colunas_datas, errors='coerce')
mapa_colunas = dict(zip(colunas_datas_dt, colunas_datas))

colunas_datas = pd.to_datetime(df_ajuste.columns[1:], errors='coerce')  # Ignora a primeira coluna
coluna_dia_compra_str = mapa_colunas.get(dia_de_compra)

# Filtrar o DataFrame com as colunas desejadas
valor_ajuste = df_ajuste.loc[df_ajuste['Assets'] == asset, coluna_dia_compra_str].values[0]
#Rendimento é igual a linha da Asset que contém o ativo somando as colunas de data
rendimento = valor_ajuste * qtd_final
rendimento



<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:9: SyntaxWarning: invalid escape sequence '\.'
C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_4204\53670784.py:9: SyntaxWarning: invalid escape sequence '\.'
  df_ajuste[colunas_datas] = df_ajuste[colunas_datas].replace('\.', '', regex=True).replace(',', '.', regex=True)


6935.8

In [5]:
from sidrapy import get_table
import pandas as pd

# Baixa a tabela 1737
data = get_table(
    table_code="1737",
    territorial_level="1",  # 1 = Brasil
    ibge_territorial_code="all",
    period="all",
    variables="all"
)

# Converte em DataFrame
df = pd.DataFrame(data)
df


,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Mês (Código),Mês,Variável (Código),Variável
1,1,Brasil,30,Número-índice,0.0000000076183,1,Brasil,197912,dezembro 1979,2266,IPCA - Número-índice (base: dezembro de 1993 =...
2,1,Brasil,,,...,1,Brasil,197912,dezembro 1979,63,IPCA - Variação mensal
3,1,Brasil,,,...,1,Brasil,197912,dezembro 1979,2263,IPCA - Variação acumulada em 3 meses
4,1,Brasil,,,...,1,Brasil,197912,dezembro 1979,2264,IPCA - Variação acumulada em 6 meses
...,...,...,...,...,...,...,...,...,...,...,...
3260,1,Brasil,2,%,0.56,1,Brasil,202503,março 2025,63,IPCA - Variação mensal
3261,1,Brasil,2,%,2.04,1,Brasil,202503,março 2025,2263,IPCA - Variação acumulada em 3 meses
3262,1,Brasil,2,%,3.55,1,Brasil,202503,março 2025,2264,IPCA - Variação acumulada em 6 meses
3263,1,Brasil,2,%,2.04,1,Brasil,202503,março 2025,69,IPCA - Variação acumulada no ano
